Dataset yang digunakan dapat didownload di: https://github.com/rizalespe/Dataset-Sentimen-Analisis-Bahasa-Indonesia atau menggunakan ***git clone*** seperti contoh dibawah ini. Folder yang di _clone_ tersimpan ke dalam folder tempat file project ini disimpan.

In [1]:
#!git clone https://github.com/rizalespe/Dataset-Sentimen-Analisis-Bahasa-Indonesia

## Install Package

**Requirement Package**:

```
1. nltk : https://www.nltk.org/
2. Sastrawi: https://github.com/sastrawi/sastrawi
3. numpy: https://numpy.org/
4. pandas: https://pandas.pydata.org/
5. sklearn: https://scikit-learn.org/stable/

```

# Import Package

In [3]:
#!pip install Sastrawi
#nltk.download('stopwords')
#nltk.download('punkt')

In [2]:
import numpy as np
import pandas as pd
import re
import pickle
from string import punctuation
import os
import json

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.model_selection import train_test_split

# ```{Utils}```

In [5]:
def process_tweet(tweet):
    
    # kumpulan stemming
    factory_stem = StemmerFactory()
    stemmer = factory_stem.create_stemmer()

    # kumpulan stopwords
    factory_stopwords = StopWordRemoverFactory()
    stopword = factory_stopwords.get_stop_words() + stopwords.words('indonesian')
  
    # menghapus kata-kata yang tidak penting seperti @, #
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'<USERNAME>', '', tweet)
    
    # tokenizer word
    tweet_tokens = word_tokenize(tweet)
    
    # membersihkan word
    tweets_clean = [stemmer.stem(word) for word in tweet_tokens if (word not in stopword and word not in punctuation)]
  
    return tweets_clean

In [6]:
def build_freqs(tweets, ys):
    
    yslist = np.squeeze(ys).tolist()
    
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

# Processing data

### Import data

In [21]:
df = pd.read_csv("data/dataset_komentar_instagram_cyberbullying.csv")

In [22]:
df.head()

,Id,Sentiment,Instagram Comment Text
0,1,negative,<USERNAME> TOLOL!! Gak ada hubungan nya kegug...
1,2,negative,Geblek lo tata...cowo bgt dibela2in balikan......
2,3,negative,Kmrn termewek2 skr lengket lg duhhh kok labil ...
3,4,negative,"Intinya kalau kesel dengan ATT nya, gausah ke ..."
4,5,negative,"hadewwwww permpuan itu lg!!!!sakit jiwa,knp ha..."


In [23]:
df.Sentiment.value_counts()

negative    200
positive    200
Name: Sentiment, dtype: int64

In [24]:
df.loc[(df.Sentiment == 'negative'),'Sentiment']=0
df.loc[(df.Sentiment == 'positive'),'Sentiment']=1

In [25]:
X = pd.DataFrame(df['Instagram Comment Text'])
y = df.Sentiment

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [27]:
X_train = X_train.values.squeeze().tolist()
X_test = X_test.values.squeeze().tolist()
y_train = np.array([y_train.values.squeeze().tolist()])
y_test = np.array([y_test.values.squeeze().tolist()])

In [28]:
X_train[0]

'Intinya kalau kesel dengan ATT nya, gausah ke anaknya juga. Kasian buat perkembangan psikis anak kedepannya. Itu orang bener bener tolol, skrg seandainya dia punya anak, terus anaknya dikatain sama orang yang benci sama dia, gimana perasaan dia ? Benci sama seseorang boleh, tapi harus tau batesnya ?? toh namanya manusia, gaakan semua jadi penyuka, pasti ada haters ??'

### Build Freqs

Cell bisa dijalankan atau langsung saja import file `freqs.json`

In [29]:
# freqs = build_freqs(X_train, y_train)

In [30]:
# check output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 2772


### save data

In [31]:
# os.makedirs(name="data", exist_ok=True)

In [32]:
#with open('data/freqs.json', 'wb') as fp:
    #pickle.dump(freqs, fp)

### load data

In [33]:
with open('data/freqs.json', 'rb') as f:
    freqs = pickle.load(f)

In [74]:
# freqs

### Process tweet

In [35]:
# test proses fungsi
print('Contoh sample positive tweet: \n', X_train[1])
print('\nContoh yang sudah di proses: \n', process_tweet(X_train[1]))

Contoh sample positive tweet: 
  <USERNAME> yaa ampun ini upil naruto,, kata2nya makin mencerminkan klo dia yg sebenarnya pecun.????pengalaman banget lu yaa. Kesian. Gua jadi prihatin. Mati aj deh lu. Ngotorin dunia manusia macam kau.??

Contoh yang sudah di proses: 
 ['yaa', 'ampun', 'upil', 'naruto', 'kata2nya', 'cermin', 'klo', 'yg', 'pecun', 'alam', 'banget', 'lu', 'yaa', 'kesi', 'gua', 'prihatin', 'mati', 'aj', 'deh', 'lu', 'ngotorin', 'dunia', 'manusia', 'kau']


# Logistic Algorithm

### Sigmoid function

In [36]:
def sigmoid(z):
    
    h = 1/(1+np.exp(-z))
    
    return h

In [37]:
# Testing fungsi 
if (sigmoid(0) == 0.5):
    print('SUCCESS!')
else:
    print('Oops!')

SUCCESS!


### Cost function

In [38]:
def linreg_cost_func(x, y, m, h):
    
    # calculate fungsi cost
    J = -1/m * (np.dot(y.T, np.log(h)) + (np.dot((1-y).T, np.log(1-h))))

    return J

### Gradient descent

In [39]:
def gradientDescent(x, y, theta, alpha, num_iters):
    
    m = x.shape[0]
    
    for i in range(0, num_iters):
        
        # feed forward
        z = np.dot(x, theta)
        h = sigmoid(z)

        # hitung cost
        J = linreg_cost_func(x, y, m, h)
        
        # update weight
        theta = theta - ((alpha/m) * (np.dot(x.T, h-y)))
        
        print(f'\rIterasi: {i+1}/{num_iters}', end='')

    J = float(J)
    return J, theta

In [47]:
# testing fungsi
np.random.seed(1)
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)


# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 2000)
print(f"\nCost setelah training adalah {tmp_J:.8f}.")

Iterasi: 2000/2000
Cost setelah training adalah 0.65476423.


# Extract Features

In [48]:
def extract_features(tweet, freqs):
    
    # process_tweet tokenizes, stems, dan hapus stopwords
    word_l = process_tweet(tweet)
    
    # inisiasi x dengan nilai 0 dalam bentuk matriks 1x3 
    x = np.zeros((1, 3)) 
    
    # inisiasi bias dengan 1
    x[0,0] = 1 
        
    for word in word_l:
        
        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1.0),0)
        
        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0.0),0)
        
    assert(x.shape == (1, 3))
    return x

In [49]:
# test fungsi
tmp1 = extract_features(X_train[5], freqs)
print(tmp1)

[[  1. 502. 605.]]


In [52]:
tmp2 = extract_features('bajingan klo kamu', freqs)
print(tmp2)

[[ 1. 13.  8.]]


# Training model

### extract features

In [53]:
X = np.zeros((len(X_train), 3))
for i in range(len(X_train)):
    X[i, :]= extract_features(X_train[i], freqs)
    print(f'\rExtract: {i+1}/{len(X_train)}', end=' ')

Extract: 320/320 

In [54]:
Y = y_train.T

### Config

In [58]:
os.makedirs(name="model", exist_ok=True)

In [59]:
config = {
    'input': X,
    'output':Y,
    'alpha':1e-8,
    'theta':np.zeros((3, 1)),
    'num_iters':2000
}

In [60]:
with open('model/model_best_linreg.pickle', 'wb') as fp:
    pickle.dump(config, fp)

### train model

In [61]:
# Apply gradient descent
J, theta = gradientDescent(config['input'], config['output'], config['theta'], config['alpha'], config['num_iters'])
print(f"\nCost setelah training adalah {J:.8f}.")

Iterasi: 2000/2000
Cost setelah training adalah 0.68604902.


In [62]:
theta

array([[ 3.20377889e-07],
       [ 2.02033706e-04],
       [-3.17695416e-04]])

### save bobot

In [63]:
with open('model/theta_best_linreg.wt', 'wb') as fp:
    pickle.dump(theta, fp)

# Test Logistic Regression

### load bobot

In [64]:
with open('model/theta_best_linreg.wt', 'rb') as f:
    theta = pickle.load(f)

In [65]:
theta

array([[ 3.20377889e-07],
       [ 2.02033706e-04],
       [-3.17695416e-04]])

In [66]:
def predict_tweet(tweet, freqs, theta):

    # extract features dari tweet dan simpan di dalam x
    x = extract_features(tweet, freqs)
    
    # buat prediksi menggunakan x dan bobot (theta)
    y_pred = sigmoid(np.dot(x, theta))
    
    return y_pred

In [67]:
# Run this cell to test your function
for tweet in ['bangsat lu', 'mantap sihh ini', 'apaan lo?', 'congrats yah k']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

bangsat lu -> 0.498910
mantap sihh ini -> 0.500051
apaan lo? -> 0.498253
congrats yah k -> 0.499653


In [68]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'bau busuk barangnya'
predict_tweet(my_tweet, freqs, theta)

array([[0.49989174]])

# Check peformance

In [69]:
def test_logistic_regression(test_x, test_y, freqs, theta):

    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        
        y_pred = predict_tweet(tweet, freqs, theta)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    accuracy = (y_hat==np.squeeze(test_y)).sum()/len(test_x)
    
    return accuracy

In [70]:
tmp_accuracy = test_logistic_regression(X_test, y_test, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.7250


# Error Analysis

In [71]:
y_test = y_test.T

In [72]:
# Some error analysis 
print('Label Predicted Tweet')
for x,y in zip(X_test,y_test):
    y_hat = predict_tweet(x, freqs, theta)
    if np.abs(y - (y_hat > 0.5)) > 0:
        print('tweet:', x)
        print('proses tweet:', process_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(process_tweet(x)).encode('ascii', 'ignore')))
        print('\n')

Label Predicted Tweet
tweet: Bagus dong.. mencari ilmu kapanpun dimanapun.. itu karena suaminya orang berpendidikan ????????????
proses tweet: ['bagus', 'dong', 'cari', 'ilmu', 'mana', 'suami', 'orang', 'didik']
1	0.49843903	b'bagus dong cari ilmu mana suami orang didik'


tweet: Beginilah pasangan suami istri yg normal.. Romantisme hanya dia dan istri yg tau. Ga perlu diumbar.. Tp dijaga. Yg suka umbar kemesraan iti kebanyakan menutupi kenyataan yg sebenarnya.
proses tweet: ['begini', 'pasang', 'suami', 'istri', 'yg', 'normal', 'romantisme', 'istri', 'yg', 'tau', 'ga', 'umbar', 'tp', 'jaga', 'yg', 'suka', 'umbar', 'mesra', 'iti', 'banyak', 'tutup', 'nyata', 'yg']
1	0.48484908	b'begini pasang suami istri yg normal romantisme istri yg tau ga umbar tp jaga yg suka umbar mesra iti banyak tutup nyata yg'


tweet: Cantik, polos, apa adanya tapi keliatan berbakat. Semoga kalau udah tenar gek jadi selebgram, kalian ga kaya2 si mba Novilda ya dek yg kehilangan arah. Tetaplah begini,jangan cepa

# Predict own tweet

In [73]:
# coba predik
my_tweet = """
kenapa loo, mau cari gara-gara
"""
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['loo', 'cari', 'gara-gara']
[[0.4997834]]
Negative sentiment
